In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Subject
For each ID in the test set, you must predict a "probability" for the TARGET variable. The file should contain a header and have the following format
-> The important thing is probability.. so actually isn't classification contest

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("../input/santander-customer-satisfaction/train.csv") 
test = pd.read_csv("../input/santander-customer-satisfaction/test.csv") 

In [ ]:
# over 370 columns in data
train.shape, test.shape

In [ ]:
#most of columns are hard to know 
np.array(train.columns.to_list()[:10])

In [ ]:
# some of values are quite big ex) imp_ent_var16_ult1, imp_op_var39_comer_ult1
# some of values are only 0 ex) ind_var2_0
# some of values are seem like percent ex) ind_var13
pd.options.display.max_columns = 380
train.describe()

In [ ]:
cols = train.columns

In [ ]:
list(set([col.split('_')[0] for col in cols]))

In [ ]:
#some columns have "special" tag on front
ind_cols = cols[[col.split('_')[0] == 'ind' for col in cols]]
imp_cols = cols[[col.split('_')[0] == 'imp' for col in cols]]
saldo_cols = cols[[col.split('_')[0] == 'saldo' for col in cols]]
delta_cols = cols[[col.split('_')[0] == 'delta' for col in cols]]
num_cols = cols[[col.split('_')[0] == 'num' for col in cols]]
len(ind_cols), len(imp_cols), len(saldo_cols), len(delta_cols), len(num_cols)

In [ ]:
#'YES' there are special tag in colunms
# 'ind' are 0 or 1
train[ind_cols].describe()

In [ ]:
#'imp' are quite big gap between min and max and most of values are 0
train[imp_cols].describe()

In [ ]:
#'saldo' similar 'imp' are quite big gap between min and max
train[saldo_cols].describe()

In [ ]:
#'delta' have same max_values
train[delta_cols].describe()

In [ ]:
#'num' num_with var45 is little bit more special than others
train[num_cols].describe()

In [ ]:
# TARGET is imblanced
train.TARGET.value_counts()

In [ ]:
# no nan value
train.isna().sum().sum()

# See each columns as I can

In [ ]:
#-9999999 seem like null value, have to replace common value '2'
train.var3.value_counts()

In [ ]:
# will do later
# train.loc[train.var3 == -999999, 'var3'] = 2
# test.loc[test.var3 == -999999, 'var3'] = 2

In [ ]:
#not bad... some people think var15 is age
train.var15.hist()

In [ ]:
# most of low_value are satisfied customers
sns.FacetGrid(train, hue= 'TARGET',height= 5).map(sns.kdeplot, 'var15').add_legend()

In [ ]:
# 99 is seem like null values
train.var36.value_counts()

In [ ]:
# 99... half and half
sns.FacetGrid(train, hue= 'TARGET',height= 5).map(sns.kdeplot, 'var36').add_legend()

In [ ]:
# most of 3 are TARGET 0
temp = train[train.var36 !=99]
sns.FacetGrid(temp, hue= 'TARGET',height= 5).map(sns.kdeplot, 'var36').add_legend()

In [ ]:
# most of are 0
train.var21.value_counts()

In [ ]:
# var21 0 is 0 but hard to guess
temp = train[train.var21 ==0]
temp.TARGET.value_counts()

In [ ]:
# most of 3 are TARGET 0
temp = train[train.var21 !=0]
sns.FacetGrid(temp, hue= 'TARGET',height= 5).map(sns.kdeplot, 'var21').add_legend()

In [ ]:
# most of are 0
train.var38.value_counts()

In [ ]:
#left-skewed ditribution... need to normalize
train.var38.hist()

In [ ]:
# look better
train.var38 = train.var38.apply(lambda x : np.log(x))
test.var38 = test.var38.apply(lambda x : np.log(x))
train.var38.hist()

In [ ]:
def prerpocessing(data):
    #var3 -999999 to 2
    data.loc[data.var3 == -999999, 'var3'] = 2
    #var38 to log
    data.var38 = data.var38.apply(lambda x : np.log(x))
    return data

In [ ]:
train = pd.read_csv("../input/santander-customer-satisfaction/train.csv") 
test = pd.read_csv("../input/santander-customer-satisfaction/test.csv") 
pre_train = prerpocessing(train)
pre_test= prerpocessing(test)

# try baseline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
lr = LogisticRegression(random_state= 86)
rc = RandomForestClassifier(random_state= 86)
xg = XGBClassifier(eval_metric= 'auc',random_state= 86)
lc = LGBMClassifier(random_state= 86)

In [ ]:
y = train.TARGET
X = train.drop(['ID','TARGET'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size = 0.2, random_state= 86)

### Logistic regreesion

In [ ]:
lr.fit(X_train,y_train)
lr_pred = lr.predict_proba(X_test)
lr_result = roc_auc_score(y_test, lr_pred[:,1])
print(lr_result)

In [ ]:
# most important are so small to remove som
lr_fi = pd.DataFrame(np.abs(lr.coef_[0]), columns = ['fi'], index=X.columns).sort_values('fi',ascending= False)
lr_fi.head(10)

### RF

In [ ]:
rc.fit(X_train, y_train)
rc_pred = rc.predict_proba(X_test)
rc_result = roc_auc_score(y_test, rc_pred[:,1])
print(rc_result)

In [ ]:
# most important columns are var38, var15
rc_fi = pd.DataFrame(rc.feature_importances_, columns = ['fi'], index=X.columns).sort_values('fi',ascending= False)
rc_fi.head(10)

### xg

In [ ]:
%%time
xg.fit(X_train, y_train)
xg_pred = xg.predict_proba(X_test)
xg_result = roc_auc_score(y_test, xg_pred[:,1])
print(xg_result)

In [ ]:
# most_columns are not important. have to remove some 
xg_fi = pd.DataFrame(xg.feature_importances_, columns = ['fi'], index=X.columns).sort_values('fi',ascending= False)
xg_fi.head(10)

### lg 

In [ ]:
%%time
lc.fit(X_train, y_train)
lc_pred = lc.predict_proba(X_test)
lc_result = roc_auc_score(y_test, lc_pred[:,1])
print(lc_result)

In [ ]:
# most_columns are not important. have to remove some 
lc_fi= pd.DataFrame(lc.feature_importances_, columns = ['fi'], index=X.columns).sort_values('fi',ascending= False)
lc_fi.head(10)

In [ ]:
lr_fi.columns = ['lr']
rc_fi.columns = ['rc']
lc_fi.columns = ['lc']
xg_fi.columns = ['xg']
fi_df = lr_fi.merge(rc_fi, right_index = True, left_index= True)
fi_df = fi_df.merge(xg_fi, right_index = True, left_index= True)
fi_df = fi_df.merge(lc_fi, right_index = True, left_index= True)

In [ ]:
#RF, lightgbm are little bit similar
fi_df.sort_values('lc',ascending= False)

### try baseline lightgbm model

In [ ]:
lc.fit(X,y)
test_X = test.drop('ID',axis= 1)
test_pred = lc.predict_proba(test_X)

In [ ]:
submission = pd.DataFrame({'ID':test.ID,'TARGET':test_pred[:,1]})
submission.to_csv('submission.csv',index = False)

In [ ]:
submission['TARGET'].sum()